In [1]:
import requests
import browsercookie
import pickle
import os
import pandas as pd
import json
import time 

NEED_INIT = False
SLEEP_INTERVAL = 10
PICKLE_PATH = "data/submissions.p"
CSV_PATH="data/submssions.csv"
submissions = {}
url = "https://leetcode.com/api/submissions/?offset={}&limit=20&lastkey="
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
HEADERS = {'User-Agent': user_agent, 'Connection': 'keep-alive'}

cj = browsercookie.chrome()

In [2]:
if os.path.exists(PICKLE_PATH):
    with open(PICKLE_PATH, "rb") as inp:
        submissions = pickle.load(inp)

In [4]:
len(submissions)

859

In [5]:
offset = 0
while True:
    req = requests.get(url.format(offset), cookies=cj, headers=HEADERS)
    records = json.loads(req.text)["submissions_dump"]

    if len(records) == 0:
        print(url.format(offset)+" does not provide any records. It may be the end.")
        break
        
    found_prev = False 
    for rec in records:
        if not rec['id'] in submissions:
            submissions[rec['id']] = rec    
        else:
            if not NEED_INIT:
                found_prev = True
                break
    if found_prev:
        print(url.format(offset)+" shows a record which we have collected. So, we don't need to proceed further.")
        break
        
    print("Loading {}-{}... We now have collected {} submissions.".format(offset,offset+19, len(submissions)))
    time.sleep(SLEEP_INTERVAL)
    offset += 20

https://leetcode.com/api/submissions/?offset=0&limit=20&lastkey= shows a record which we have collected. So, we don't need to proceed further.


In [6]:
with open(PICKLE_PATH, "wb") as oup:
    pickle.dump(submissions, oup)
    
print("{} records have been stored at '{}'".format(len(submissions), PICKLE_PATH))

862 records have been stored at 'data/submissions.p'


In [7]:
df = pd.DataFrame([x[1] for x in submissions.items()])
df.head()

,id,lang,time,timestamp,status_display,runtime,url,is_pending,title,memory,code,compare_result
0,378488337,javascript,"1 hour, 2 minutes",1596997036,Accepted,152 ms,/submissions/detail/378488337/,Not Pending,Number of Sub-arrays With Odd Sum,58.5 MB,/**\n * @param {number[]} arr\n * @return {num...,1111111111111111111111111111111111111111111111...
1,378478285,javascript,"1 hour, 28 minutes",1596995516,Accepted,88 ms,/submissions/detail/378478285/,Not Pending,Count Number of Teams,36.6 MB,/**\n * @param {number[]} rating\n * @return {...,1111111111111111111111111111111111111111111111...
2,378474658,javascript,"1 hour, 37 minutes",1596994944,Accepted,76 ms,/submissions/detail/378474658/,Not Pending,Reformat Date,36.9 MB,/**\n * @param {string} date\n * @return {stri...,1111111111111111111111111111111111111111111111...
3,378472770,python,"1 hour, 42 minutes",1596994654,Accepted,36 ms,/submissions/detail/378472770/,Not Pending,Pancake Sorting,12.7 MB,class Solution(object):\n def pancakeSort(s...,1111111111111111111111111111111111111111111111...
4,378466216,javascript,"1 hour, 59 minutes",1596993630,Accepted,72 ms,/submissions/detail/378466216/,Not Pending,Kth Missing Positive Number,37.4 MB,/**\n * @param {number[]} arr\n * @param {numb...,1111111111111111111111111111111111111111111111...


In [8]:
df.to_csv(CSV_PATH, index=False)
print("A data frame file has been stored at '{}'".format(CSV_PATH))

A data frame file has been stored at 'data/submssions.csv'
